# SQL Optimizer in SQL
This notebook accompanies the extra [homework assignment](README.md) for CS286A, Fall 2017. Please see that document for instructions on installation and setup.

First, load support for the %sql magic in Jupyter, and log into the database.
You will have to do this every time you start the kernel in this Jupyter notebook.

In [1]:
%load_ext sql
%sql postgresql://vagrant:vagrant@:5432/optimizer

u'Connected: vagrant@optimizer'

## Setup 1. Create metadata tables for our optimizer
This section sets up all the information needed for our optimizer as a set of metadata tables. You can see a [diagram of the full schema here.](img/metaschema.png)

### Create a separate schema to hold the optimizer's tables and views
SQL schemas are like filesystem directories, except they cannot be nested. Here we will separate out the optimizer tables and views by creasing a new schema, `sqlopt`, and setting the database `search_path` to default to `sqlopt`. All the tables and views we define below will be under that schema. By default, in postgresql the search path starts with `public`. So if you connect to this database from psql you will have to reference the optimizer tables with the prefix `sqlopt.`, as in `SELECT * FROM sqlopt.catalog_tables`. More info on schemas is in the [PostgreSQL documentation](https://www.postgresql.org/docs/current/static/ddl-schemas.html).

In [2]:
%%sql
DROP SCHEMA IF EXISTS sqlopt CASCADE;
CREATE SCHEMA sqlopt;
SET search_path to sqlopt,public;

-- Postgres doesn't include a built-in array sorting function.
-- We define one here. You do not need to understand this in any detail.
CREATE OR REPLACE FUNCTION array_sort(anyarray)
RETURNS anyarray AS $$
  SELECT ARRAY(SELECT unnest($1) ORDER BY 1)
$$ LANGUAGE sql;

-- Test the sorting function
CREATE TABLE test_array(rownumber integer, words text[]);
INSERT INTO test_array VALUES (1, '{the, quick, brown}'), (2, '{four, score, and, seven}');
SELECT rownumber, array_sort(words) AS sorted FROM test_array;

Done.
Done.
Done.
Done.
Done.
2 rows affected.
2 rows affected.


rownumber,sorted
1,"[u'brown', u'quick', u'the']"
2,"[u'and', u'four', u'score', u'seven']"


### System: metadata about the hardware and software configuration
The only state we'll keep about our "hardware configuration" is the size of the buffer, the number of buffers available for our query operators (sorts, hash joins, etc.) and "CPU factor" in the Selinger cost model. For our software configuration, we'll keep the type of our join methods. 

System information tables have the prefix `system_`.

In [3]:
%%sql
DROP TABLE IF EXISTS system_hardware CASCADE;
DROP TABLE IF EXISTS system_join_methods CASCADE;

--
-- Machine Config
--
-- bufpool is size of the buffer pool
-- qbufs is the number of pages of memory allocated to each operator
-- cpufactor is the constant from Selinger's paper
CREATE TABLE system_hardware(bufpool integer, qbufs integer, cpufactor float);
INSERT INTO system_hardware VALUES (10, 5, .001);

--
-- Query Operators
--
-- right_index_requirement is TRUE if the join algorithm requires the
-- join predicate to match an index.
CREATE TABLE system_join_methods(name text PRIMARY KEY, right_index_requirement bool);
INSERT INTO system_join_methods VALUES 
  ('BNLJ', false), 
  ('INLJ', true), 
  ('Hash', false), 
  ('Merge', false);

Done.
Done.
Done.
1 rows affected.
Done.
4 rows affected.


[]

### Catalog metadata: tables that store information about tables.
We'll need to keep track of the user's schema and basic statistics about their data. For this implementation, we'll just use the simple Selinger statistics.

All of our catalog tables with start with the prefix `catalog_`.

In [4]:
%%sql
DROP TABLE IF EXISTS catalog_tables CASCADE;
DROP TABLE IF EXISTS catalog_columns CASCADE;
DROP TABLE IF EXISTS catalog_access_methods CASCADE;

-- tables and columns
CREATE TABLE catalog_tables(table_name text PRIMARY KEY, ntups integer, npages integer);
CREATE TABLE catalog_columns(table_name text, column_id integer, name text, type text, num_vals integer,
                        lo integer, hi integer,
                        PRIMARY KEY (table_name, column_id),
                        FOREIGN KEY (table_name) REFERENCES catalog_tables);

-- access methods: current limitation: single-column search keys
CREATE TABLE catalog_access_methods(am_id integer, table_name text, 
                            column_id integer, type text,
                            PRIMARY KEY (am_id),
                            FOREIGN KEY (table_name, column_id) 
                            REFERENCES catalog_columns);

Done.
Done.
Done.
Done.
Done.
Done.


[]

### Parser metadata: information about the query to be optimized.
Here we define tables that hold information about the query to be optimized. You should assume these tables are populated by a parser for SQL; for testing we'll populate them by hand.

We only handle simple single-query blocks of the form
```SQL
   SELECT *
     FROM <query_tables>
    WHERE <T1.col> <op> <constant> (selections)
      AND <T1.col> = <T2.col> (two-relation equijoins)
```
All the query-specific state has the `query_` prefix.

In [5]:
%%sql

DROP TABLE IF EXISTS query_tables CASCADE;
DROP TABLE IF EXISTS query_predicates CASCADE;

-- tables referenced
CREATE TABLE query_tables(table_name text PRIMARY KEY,
                          FOREIGN KEY (table_name) REFERENCES catalog_tables);

-- Assuming the WHERE clause is in CNF, each of the conjuncts
-- is called a "predicate" (Selinger calls it a "Boolean Factor").
-- We currently do not handle OR or NOT predicates, only conjunctions of single-table
-- and two-table predicates.
-- Note the CHECK constraint that ensures each row holds either single-table or 
-- two-table information.
CREATE TABLE query_predicates(
    pred_id integer PRIMARY KEY, 
    table_left text NOT NULL, column_left integer NOT NULL,
    operator char(2), constant integer,        -- for selection predicates
    table_right text, column_right integer, -- for join predicates
    FOREIGN KEY (table_left, column_left) REFERENCES catalog_columns,
    FOREIGN KEY (table_right, column_right) REFERENCES catalog_columns,
    FOREIGN KEY (table_left) REFERENCES query_tables,
    FOREIGN KEY (table_right) REFERENCES query_tables,
    
    CHECK ((constant IS NULL AND operator IS NULL AND table_right IS NOT NULL AND column_right IS NOT NULL) 
           OR 
           (constant IS NOT NULL AND operator IS NOT NULL AND table_right IS NULL AND column_right IS NULL))
);

Done.
Done.
Done.
Done.


[]

### The output of the optimization algorithm.
Here we register the schema of the output you need to generate from the Selinger algorithm. You should understand the structure of this table and its constraints well before you try to implement the optimizer algorithm.  This is a typical relational representation of a binary tree, in which each row represents a node in the tree with a unique `node_id`, and references other nodes. Here we're linked upward and downward: we have references to a `lhs` (left-hand-side) child, an `rhs` child, and a `parent` in each node. We also have explicit ordering among siblings captured in `sibling_id`, and a "level" of the node within the tree (leaves are at level 1, and parents are 1 level higher than their highest child.)

Metadata for the optimizer algorithms is prefixed with `opt_`.

In [6]:
%%sql
--
-- Optimizer state
--
DROP TABLE IF EXISTS opt_chosen_plan;

-- The final chosen plan: your code needs to populate this table correctly.
CREATE TABLE opt_chosen_plan 
(
    node_id integer PRIMARY KEY,         -- id of this node
    level integer,       -- level in the tree (scan is 1)
    tables text[],       -- array of tables in this subtree
    am_id integer,       -- level=1: access method ID
    type text,           -- level=1: access method type
    top_join text,       -- level>1: join method
    lhs integer,         -- level>1: pid of left child
    rhs integer,         -- level>1: pid of right child,
    cost float,          -- cost
    npages integer,      -- number of pages of output
    ntups integer,       -- number of tuples of output
    order_table text, 
    order_column integer,  -- this column and the preceding define the ordering key of the output
    parent integer,      -- pid of the parent node
    sibling_id integer,  -- position in order of children of this parent (1 = left, 2 = right),
    FOREIGN KEY (am_id) REFERENCES catalog_access_methods,
    FOREIGN KEY (top_join) REFERENCES system_join_methods,
    FOREIGN KEY (lhs) REFERENCES opt_chosen_plan,
    FOREIGN KEY (rhs) REFERENCES opt_chosen_plan,
    FOREIGN KEY (order_table, order_column) REFERENCES catalog_columns,
    FOREIGN KEY (parent) REFERENCES opt_chosen_plan,
    
    CHECK (   (level = 1 AND am_id IS NOT NULL AND type IS NOT NULL)
           OR (level > 1 AND top_join IS NOT NULL AND lhs IS NOT NULL and rhs IS NOT NULL))
);

Done.
Done.


[]

## Setup 2: Populating the Metadata
### Create Boating Club database
Here we define and populate an instance of the boating club database from the class examples. The actual data tables will be in the `public` schema. Since our `search_path` was set to have `sqlopt` first, we explicitly specify the `public` schema here.

This is the first time we embed SQL in Python code. Note the use of the one-line (`%sql`) syntax for `ipython-sql`, and the way we pass Python variable values into SQL queries via the `:` prefix.

In [7]:
import os

%sql DROP TABLE IF EXISTS public.sailors CASCADE;
%sql DROP TABLE IF EXISTS public.boats CASCADE;
%sql DROP TABLE IF EXISTS public.reserves CASCADE;
%sql CREATE TABLE public.sailors(sid integer PRIMARY KEY, sname text, rating integer);
%sql CREATE TABLE public.boats(bid integer PRIMARY KEY, bname text, color text);
%sql CREATE TABLE public.reserves(rid integer PRIMARY KEY, sid integer, bid integer, rdate date, \
                      FOREIGN KEY (sid) REFERENCES sailors,\
                      FOREIGN KEY (bid) REFERENCES boats);
%sql CREATE INDEX ON public.reserves(bid);
%sql CREATE INDEX ON public.reserves(sid);

cwd = os.getcwd()
sailor_path = cwd + '/sailors.csv'
boat_path = cwd + '/boats.csv'
reserve_path = cwd + '/reserves.csv'
%sql COPY sailors FROM :sailor_path WITH (FORMAT csv);
%sql COPY boats FROM :boat_path WITH (FORMAT csv);
%sql COPY reserves FROM :reserve_path WITH (FORMAT csv);   
%sql ANALYZE;

Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
100 rows affected.
117 rows affected.
100000 rows affected.
Done.


[]

### Populate the metadata: information about the class Boating Club database
Here we copy metadata from the PostgreSQL catalog into our own catalog tables.

In [8]:
%%sql
INSERT INTO catalog_tables 
SELECT table_name, NULL::integer, NULL::integer
  FROM information_schema.tables
 WHERE table_schema = 'public';
    
INSERT INTO catalog_columns
SELECT table_name, ordinal_position, column_name, data_type, NULL::integer
  FROM information_schema.columns
 WHERE table_schema = 'public';

3 rows affected.
10 rows affected.


[]

### Populate access methods.
Here we populate access methods for our tables:
- We assume there is a heap file for each table
- We also query PostgreSQL to see what indexes have been created, assuming each index is of type 'btree'. Note that PostgreSQL automatically builds an index on the primary key of a table.
This is the first time we use a SQL sequence number generator to autogenerate unique ids. You can read more about SQL sequences [in the PostgreSQL manual](https://www.postgresql.org/docs/current/static/sql-createsequence.html).

In [9]:
%%sql
-- Sequence generator for access method ids
DROP SEQUENCE IF EXISTS catalog_am_seq CASCADE;
CREATE SEQUENCE catalog_am_seq;

INSERT INTO catalog_access_methods
SELECT nextval('catalog_am_seq'), table_name, NULL, 'heap'
  FROM catalog_tables;
    
INSERT INTO catalog_access_methods
SELECT nextval('catalog_am_seq'), t.relname, a.attnum, 'btree'
  FROM
    pg_class t,
    pg_class i,
    pg_index ix,
    pg_attribute a,
    pg_namespace ns
 WHERE
    t.oid = ix.indrelid
    and i.oid = ix.indexrelid
    and a.attrelid = t.oid
    and a.attnum = ANY(ix.indkey)
    and t.relkind = 'r'
    and t.relnamespace = ns.oid
    and ns.nspname = 'public';

Done.
Done.
3 rows affected.
5 rows affected.


[]

### The next cell represents a parsed query, stored as a bunch of metadata.
Here we populate the query tables with parsed output for the following SQL query:
```sql
SELECT * 
  FROM sailors S, reserves R, boats B
 WHERE S.rating > 5 AND B.bid = 100 AND S.sid = R.sid AND R.bid = B.bid;
```

In [10]:
%%sql

INSERT INTO query_tables VALUES ('sailors'), ('boats'), ('reserves');
INSERT INTO query_predicates VALUES 
    (1, 'sailors', 3, '>', 5, NULL, NULL), (2, 'boats', 1, '=', 100, NULL, NULL), 
    (4, 'sailors', 1, NULL, NULL, 'reserves', 2), (5, 'reserves', 3, NULL, NULL, 'boats', 1);

3 rows affected.
4 rows affected.


[]

## Homework Part 1: Optimizer cost and selectivity formulae
Your first challenge is to define the optimizer cost formulae as PostgreSQL user-defined functions (UDFs). These functions should return type `numeric`, and can be implemented as simple SQL expressions that return a single row with a single column.  More information on PostgreSQL UDFs can be found [in the manual](https://www.postgresql.org/docs/current/static/xfunc-sql.html).

For simplicity we will not distinguish clustered and unclustered indexes; assume the cost of an index scan is the unclustered cost.

In [11]:
%%sql
DROP FUNCTION IF EXISTS SeqScanCost(integer, integer) CASCADE;
DROP FUNCTION IF EXISTS IndexScanCost(integer, integer);
DROP FUNCTION IF EXISTS HashJoinCost(integer, integer, integer, integer, integer);
DROP FUNCTION IF EXISTS SortMergeJoinCost(integer, integer, integer, integer, integer);
DROP FUNCTION IF EXISTS BNLJCost(integer, integer, integer, integer, integer);
DROP FUNCTION IF EXISTS INLJCost(integer, integer, integer, integer, integer, integer);
DROP FUNCTION IF EXISTS SingleColumnSelectivity(char(2), integer, integer, integer);
DROP FUNCTION IF EXISTS TwoColumnEqSelectivity(integer, integer);

--
-- Scans
--
CREATE FUNCTION SeqScanCost(npages integer, ntups integer) RETURNS numeric AS $$
  -- WE ARE GIVING YOU THIS ONE FOR FREE :-)
  SELECT (npages + H.cpufactor*ntups)::numeric
    FROM system_hardware H
   LIMIT 1
$$ LANGUAGE SQL;

-- Unclustered cost only for now
CREATE FUNCTION IndexScanCost(npages integer, ntups integer) 
RETURNS numeric AS $$
    -- YOUR CODE HERE
    -- traverse cost is ignored
   SELECT (ntups + H.cpufactor*ntups)::numeric
    FROM system_hardware H
   LIMIT 1
$$ LANGUAGE SQL;


--
-- Joins
-- 
CREATE FUNCTION HashJoinCost(qbufs integer, lhspages integer, lhstups integer, 
                         rhspages integer, rhstups integer) RETURNS numeric AS $$
    -- YOUR CODE HERE
    -- we choose the smaller number of pages among the two sides
    SELECT ((CEIL(LOG(qbufs-1,CEIL(LEAST(lhspages,rhspages)/(qbufs-2)::decimal)::decimal))*2+1)*(lhspages+rhspages) + H.cpufactor*(lhstups+rhstups))::numeric
    FROM system_hardware H
    LIMIT 1
$$ LANGUAGE SQL;

CREATE FUNCTION SortMergeJoinCost(qbufs integer, lhspages integer, lhstups integer, 
                         rhspages integer, rhstups integer) RETURNS numeric AS $$
    -- YOUR CODE HERE
    SELECT (lhspages+rhspages+CEIL(LOG(qbufs-1,CEIL(lhspages/qbufs::decimal)::decimal))+CEIL(LOG(qbufs-1,CEIL(rhspages/qbufs::decimal)::decimal))+H.cpufactor*(lhstups+rhstups))::numeric
    FROM system_hardware H
    LIMIT 1
$$ LANGUAGE SQL;

DROP FUNCTION IF EXISTS BNLJCost(integer, integer, integer, integer, integer);
CREATE FUNCTION BNLJCost(qbufs integer, lhspages integer, lhstups integer, 
                         rhspages integer, rhstups integer) RETURNS numeric AS $$
    -- YOUR CODE HERE
    -- we try both side use the smaller one as the cost
    SELECT (LEAST(CEIL(lhspages/(qbufs-2)::decimal)*rhspages+lhspages, CEIL(rhspages/(qbufs-2)::decimal)*lhspages+rhspages)+H.cpufactor*(lhstups+rhstups))::numeric
    FROM system_hardware H
    LIMIT 1             
$$ LANGUAGE SQL;

CREATE FUNCTION INLJCost(qbufs integer, lhspages integer, lhstups integer, 
                         rhspages integer, rhstups integer, rhs_numvals integer) 
                RETURNS numeric AS $$
    -- YOUR CODE HERE
    SELECT (lhspages+(lhspages*lhstups)/rhs_numvals + H.cpufactor*(lhstups+rhstups))::numeric
    FROM system_hardware H
    LIMIT 1
$$ LANGUAGE SQL;
--

-- This function calculate the sort cost for sort merge cost; it pass in ntups and order_column;
-- If order_column is not null means there is interesting order and already sorted so set it zero, otherwise we
-- use quick sort to it is nlog(n)
-- Sort COST
DROP FUNCTION IF EXISTS SortCost(integer,integer);
CREATE FUNCTION SortCost(ntups integer, order_column integer) RETURNS numeric AS $$
  SELECT CASE WHEN order_column is not null THEN (0)::numeric
              ELSE (LOG((2)::numeric, ntups)*ntups)::numeric END AS cost
$$ LANGUAGE SQL;


--
-- SingleColumnSelectivity
--
CREATE FUNCTION SingleColumnSelectivity(op char(2), constant integer, bin_lo integer, bin_hi integer) 
        RETURNS numeric AS $$
    -- YOUR CODE HERE
    SELECT CASE
    WHEN op = '=' THEN 1/(bin_hi - bin_lo + 1)::numeric
    WHEN op = '>' THEN (bin_hi - constant)/(bin_hi - bin_lo + 1)::numeric
    WHEN op = '>=' THEN (bin_hi - constant + 1)/(bin_hi - bin_lo + 1)::numeric
    WHEN op = '<' THEN (constant - bin_lo)/(bin_hi - bin_lo + 1)::numeric
    WHEN op = '<=' THEN (constant - bin_lo + 1)/(bin_hi - bin_lo + 1)::numeric
    WHEN op = '!=' THEN (bin_hi - bin_lo)/(bin_hi - bin_lo + 1)
    END
$$ LANGUAGE SQL;
    
--
-- TwoColumnEqSelectivity
--
CREATE FUNCTION TwoColumnEqSelectivity(lhs_height integer, rhs_height integer)
       RETURNS numeric AS $$
    -- YOUR CODE HERE
    -- choose the highest distint numbers as denominator
    SELECT (1/GREATEST(lhs_height, rhs_height)::numeric)::numeric
$$ LANGUAGE SQL;

Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

## Homework Part 2: Populate the Optimizer Statistics
You are responsible to write queries to gather the statistics about tables and columns. 

This involves two basic tasks:
- Get the number of tuples and pages for each table. Assume that each page holds 4KB. Assume that integers are 4 bytes long, floats and dates are 8 bytes long, and strings are as many bytes as their length.
- gather distinct values, min and max (1-bucket histogram) for each integer column. For non-integer columns, min and max are set to NULL

Note that you'll have to run a query over each table to compute `ntups` and `npages`, and over each column of each table to compute `num_vals`, `lo` and `hi`. This is not possible in SQL, since SQL does not allow variables in the `FROM` clause. (That would be [second-order logic](https://en.wikipedia.org/wiki/Second-order_logic)). To achieve this you will have to embed SQL in Python, and pass Python variable values into your SQL. Be careful with your use of `$` vs. `:` in `ipython-sql`!

We've given you the SQL code to look up npages in the PostgreSQL catalog; you need to write the SQL code to compute `ntups`, and the column statistics. You should *not* take the statistics from the PostgreSQL catalog, you should compute them directly from the user tables. 



In [12]:
# we get column and type from catalo_columns table and chek if the type is integer or text. For text, it's all null.
# For integer, we plug in all the data

tablenames = %sql select table_name from catalog_tables;
for t in tablenames:
    tname = t[0]
    %sql UPDATE catalog_tables \
            SET npages = (SELECT relpages \
                            FROM pg_class r, pg_namespace ns \
                           WHERE r.relnamespace = ns.oid \
                             AND r.relname = :tname \
                             AND ns.nspname = 'public') \
         WHERE table_name = :tname;
    %sql UPDATE catalog_tables \
            SET ntups = (SELECT count(*) \
                           FROM $tname) \
         WHERE table_name = :tname;
    colname = %sql select name, type from catalog_columns where table_name = :tname;
    for c in colname:
        cname = c[0]
        ctype = c[1]
        if c[1] == unicode('integer'):
            % sql UPDATE catalog_columns \
                     SET num_vals = (SELECT count(DISTINCT $cname) \
                                     FROM $tname)\
                  WHERE table_name = :tname and name = :cname; 
            % sql UPDATE catalog_columns \
                     SET lo = (SELECT MIN($cname) \
                               FROM $tname)\
                  WHERE table_name = :tname and name = :cname; 
            % sql UPDATE catalog_columns \
                     SET hi = (SELECT MAX($cname) \
                               FROM $tname)\
                  WHERE table_name = :tname and name = :cname; 
        else:
            %sql UPDATE catalog_columns \
                    SET num_vals = NULL \
                 WHERE table_name = :tname and name = :cname;
            % sql UPDATE catalog_columns \
                     SET lo = NULL \
                  WHERE table_name = :tname and name = :cname; 
            % sql UPDATE catalog_columns \
                     SET hi = NUll \
                  WHERE table_name = :tname and name = :cname;
    
# YOUR CODE GOES HERE

3 rows affected.
1 rows affected.
1 rows affected.
3 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
3 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
4 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


## Homework Part 3: Dynamic Programming
Now that we have all our state set up, we can implement the core algorithm in SQL. 

First you will want to define your dynamic programming table and any ancillary tables or views you want to keep the state of the algorithm.

Hint: you may want to use the `CREATE SEQUENCE` feature to generate unique IDs for subplans found (and later reused) during dynamic programming.


In [13]:
%%sql
-- Your DDL statements go here
DROP SEQUENCE IF EXISTS anc_am_seq CASCADE;
CREATE SEQUENCE anc_am_seq;
    
DROP TABLE IF EXISTS opt_bestplans CASCADE;
CREATE TABLE opt_bestplans(
    pid integer PRIMARY KEY,
    table_name text,
    am_id integer,
    type text,
    cost float,
    npages integer,
    ntups integer,
    order_table text,
    order_column integer,
    level integer,
    
    FOREIGN KEY (am_id) REFERENCES catalog_access_methods,
    FOREIGN KEY (order_table, order_column) REFERENCES catalog_columns
);


Done.
Done.
Done.
Done.


[]

Now you can write the DML queries for the dynamic program. Take it in two parts: base case and inductive case.

### Base case: find best single-table access plans
Here you need to write SQL to populate the dynamic programming table with level-1 information on scan subplans--both heap scans and index scans. We'll consider every heap scan, and any indexscan where the index key matches a column referenced in the query predicates.

We have given you a set of view skeletons to break the problem into pieces.

In [14]:
%%sql

-- The general idea of our code is to calclulate the accumulated cost for each level and for the final level,
-- we select the cheapest cost plan as the tree root. We did not keep updating the sibling and parent information
-- for each level, since after we get the final best plan, we can back track its children through recursion using 
-- pid and contruct the whole tree.

-- Extract all table_name references in predicates that could benefit from an index. 
-- There are two types of predicates:
-- 1) For predicates involving only one table, we want to know the 
--    table, column_id, selection operator and the constant.
-- 2) For predicates representing joins, we want to know tables 
--    and column_ids, as well as the join operator. Note that we're interested
--    in indexes both for the rhs and lhs of the predicate!
DROP VIEW IF EXISTS opt_index_refs CASCADE;
CREATE VIEW opt_index_refs(table_name, column_id, operator, constant) AS (
    -- YOUR CODE GOES HERE
    
    -- First, choose all predicates (joins decomposite to two separate predicates) and set join operator as '><'
    -- Second, join with catalog_access_methods to get columns might have index
    
    SELECT A.table_name, A.column_id, A.operator, A.constant
    FROM ((SELECT table_left AS table_name, column_left AS column_id, CASE WHEN operator IS NOT NULL THEN operator
                                                                                           ELSE '><' END AS operator, constant
    FROM query_predicates)
    UNION
    (SELECT table_right AS table_name, column_right AS column_id, CASE WHEN operator IS NOT NULL THEN operator
                                                                                           ELSE '><' END As operator, constant
    FROM query_predicates
    WHERE table_right IS NOT NULL)) AS A, catalog_access_methods AS B
    WHERE A.table_name = B.table_name AND A.column_id = B.column_id
);

-- Now combine the results of the previous view with rows that capture the case of 
-- heap scans that do not match a predicate.
-- Same schema as the previous view, but heap scans will have NULL info for 
-- columns, operators, and constants.
DROP VIEW IF EXISTS opt_table_col_refs CASCADE;
CREATE VIEW opt_table_col_refs(table_name, column_id, operator, constant) AS (
    -- YOUR CODE GOES HERE
    
    -- find heap file scan table in catalog_access_methods 
    -- union with opt_index_refs
    
    (SELECT * FROM opt_index_refs)
    UNION
    (SELECT table_name, NULL AS column_id, NULL AS operator, NULL AS constant
    FROM catalog_access_methods
    WHERE type = 'heap')
);

-- Now find all the access methods corresponding to the references in the previous view.
DROP VIEW IF EXISTS opt_scan_access_methods CASCADE;
CREATE VIEW opt_scan_access_methods(table_name, column_id, am_id, type, npages, ntups, 
                                    order_column, operator, constant) AS (
    -- YOUR CODE GOES HERE
    -- join VIEW opt_table_refs, TABLE catalo_access_methods, TABLE catalog_tables
    -- on table_name, column_id
    -- select the interested attributes
    -- SET order_column as column_id if interesting order exists
    --                               else SET NULL
    -- UNION heap file 
    
    (SELECT  A.table_name, A.column_id, B.am_id, B.type, C.npages, C.ntups, CASE WHEN A.operator = '><' THEN A.column_id 
                                                                                                         ELSE NULL END AS order_column, A.operator, A.constant
    FROM opt_table_col_refs A, catalog_access_methods B, catalog_tables C
    WHERE A.table_name = B.table_name AND A.column_id = B.column_id AND A.table_name = C.table_name)
    UNION 
    (SELECT  A.table_name, A.column_id, B.am_id, B.type, C.npages, C.ntups, CASE WHEN A.operator = '><' THEN A.column_id 
                                                                                                         ELSE NULL END AS order_column, A.operator, A.constant
    FROM opt_table_col_refs A, catalog_access_methods B, catalog_tables C
    WHERE A.table_name = B.table_name AND A.table_name = C.table_name AND B.type = 'heap' AND A.column_id IS NULL
    )
    
);

-- Compute costs for all full-scan access methods:
-- these are scans that return every tuple in the table. We should consider both 
-- heap and index access methods, as both types can return all tuples. The index
-- scans will produce interesting orders based on their search key.
DROP VIEW IF EXISTS opt_scan_AM_costs;
CREATE VIEW opt_scan_AM_costs(table_name, column_id, am_id, type, cost, npages, ntups, 
                              order_table, order_column, operator, constant) AS (
    -- YOUR CODE GOES HERE

    SELECT table_name, column_id, am_id, type, CASE WHEN operator IS NOT NULL THEN IndexScanCost(npages, ntups)
                                                    ELSE SeqScanCost(npages, ntups) END AS cost
    , npages, ntups, CASE WHEN A.order_column IS NOT NULL THEN table_name 
                                                          ELSE NULL END AS order_table, order_column, operator, constant
    FROM opt_scan_access_methods AS A

    
);

-- Compute costs for index access methods that match a query predicate.
-- Cost formulas for indexes need to take predicate selectivity into account, 
-- since they only scan a fraction of the index and heap file. Again, these
-- return interesting orders.
DROP VIEW IF EXISTS opt_index_AM_costs CASCADE;
CREATE VIEW opt_index_AM_costs(table_name, column_id, am_id, type, cost, npages, ntups,
                               order_table, order_column) AS (
    -- NEW VERSION WITHOUT seletivity in join predicates
    SELECT A.table_name, A.column_id, A.am_id, A.type, CASE WHEN A.operator = '><' THEN A.cost
                                                            WHEN A.operator IS NULL THEN A.cost
                                                            ELSE A.cost * SingleColumnSelectivity(operator, constant, B.lo, B.hi) END AS cost
    , npages, ntups, A.order_table, A.order_column
    FROM opt_scan_AM_costs A, catalog_columns B
    WHERE A.table_name = B.table_name AND A.column_id = B.column_id
    
    -- OLD VERSION
    -- SELECT A.table_name, A.column_id, A.am_id, A.type, CASE WHEN A.operator = '><' THEN A.cost * TwoColumnEqSelectivity(B.num_vals, B.num_vals)
    --                                                                                     ELSE A.cost * SingleColumnSelectivity(operator, constant, B.lo, B.hi) END AS cost
    -- , npages, ntups, A.order_table, A.order_column
    -- FROM opt_scan_AM_costs A, catalog_columns B
    -- WHERE A.table_name = B.table_name AND A.column_id = B.column_id
    -- OLD VERSION ENDS
    
);

-- Combine all of the level-1 access methods and their cost estimates
-- in a single view.
DROP VIEW IF EXISTS opt_all_scan_access_methods CASCADE;
CREATE VIEW opt_all_scan_access_methods(table_name, am_id, type, cost, npages, ntups,
                                        order_table, order_column) AS (
    -- YOUR CODE GOES HERE
    (SELECT table_name, am_id, type, cost, npages, ntups, order_table, order_column 
     FROM opt_index_AM_costs
    )
    UNION 
    (SELECT A.table_name, A.am_id, A.type, A.cost
    , npages, ntups, NULL, A.order_column
    FROM opt_scan_AM_costs A
    WHERE A.type = 'heap'
    )
);

-- Assign a unique ID to each AM.
-- We materialize this table_name so that each access method (subplan)
-- is assigned a persistent unique ID from a SQL sequence generator.
DROP TABLE IF EXISTS opt_numbered_scan_AMs CASCADE;
CREATE TABLE opt_numbered_scan_AMs(pid, table_name, am_id, type, cost, npages, ntups,
                                   order_table, order_column) AS (
    -- YOUR CODE GOES HERE
    SELECT nextval('anc_am_seq') AS pid, *
    FROM opt_all_scan_access_methods
    
);

-- Find the minimum-cost plan for each interesting order (including NULL). You
-- will need to use an "argmin" style query pattern (see HW1).
-- Insert the results (with pids) into opt_bestplans, with level = 1

INSERT INTO opt_bestplans 
SELECT *
FROM(
    WITH min_table 
    AS (SELECT MIN(cost) AS minCost
        FROM opt_numbered_scan_AMs
        GROUP BY table_name
       )

    (SELECT * ,1 AS level
     FROM opt_numbered_scan_AMs
     WHERE order_table IS NOT NULL)
     UNION
    (SELECT pid, table_name, am_id, type, cost, npages, ntups, order_table, order_column, 1 AS level
     FROM min_table, opt_numbered_scan_AMs A
     WHERE min_table.minCost = A.cost)
    ) AS A;

Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
8 rows affected.
7 rows affected.


[]

### Inductive Case: form recursively larger join trees
In this cell we consider all ways of joining the n-1-way joins with the level-1 scans. Be sure to keep plans for all interesting orders! (Note that we are not handling GROUP BY or ORDER BY in this homework, so the only interesting orders related to joins.)

For each plan you consider, you need to compute cost, and use selectivity estimates to compute the output npages and ntups. You also need to track the output ordering based on the input. Currently we only track one column as an interesting order; for sort-merge join you can pick either of the join columns.

Make sure you consider "commuting" the join predicates: i.e. if you have a predicate of the form `R.x = S.y` you should also consider plans in which `R.x` is the right child of a join.

You can implement the induction by looping over values of n in Python, or by using a SQL [`WITH RECURSIVE`](https://www.postgresql.org/docs/9.6/static/queries-with.html) query.

In [15]:
%%sql

-- YOUR CODE GOES HERE

-- we create a view called query_join_predicates and extract all the join predicates from query_predicates 
-- into this view

DROP VIEW IF EXISTS query_join_predicates CASCADE;
CREATE VIEW query_join_predicates AS(
    SELECT pred_id, table_left, column_left, table_right, column_right
    FROM query_predicates
    WHERE operator IS NULL
);

-- This is used to update output_size of three TABLES

-- USE TABLE catalog_columns AND TABLE query_predicates
DROP VIEW IF EXISTS pass1_output_size CASCADE;
CREATE VIEW pass1_output_size AS(
    SELECT A.table_name, CEIL(A.npages::decimal*SingleColumnSelectivity(B.operator, B.constant, A.lo, A.hi)) AS npages,
    CEIL(A.ntups::decimal*SingleColumnSelectivity(B.operator, B.constant, A.lo, A.hi)) AS ntups
    FROM (SELECT C.*, D.ntups, D.npages
            FROM catalog_columns C 
            JOIN catalog_tables D
              ON C.table_name = D.table_name
         ) AS A 
                JOIN query_predicates B
                ON A.table_name = B.table_left AND A.column_id = B.column_left
    WHERE  B.operator IS NOT NULL

);

UPDATE opt_bestplans
SET npages = ((SELECT npages FROM pass1_output_size B WHERE opt_bestplans.table_name = B.table_name))::integer,
    ntups = ((SELECT ntups FROM pass1_output_size B WHERE opt_bestplans.table_name = B.table_name))::integer
FROM pass1_output_size B
WHERE opt_bestplans.table_name = B.table_name;

Done.
Done.
Done.
Done.
4 rows affected.


[]

In [16]:
%%sql

-- We crested a TABLE stores the BESTPLANS on each PASS(called opt_bestpassplans)
-- HERE ntups and npages should be output size


DROP TABLE IF EXISTS opt_bestpassplans CASCADE;
CREATE TABLE opt_bestpassplans(    
    
    node_id integer PRIMARY KEY,         -- id of this node
    level integer,       -- level in the tree (scan is 1)
    tables text[],       -- array of tables in this subtree
    am_id integer,       -- level=1: access method ID
    type text,           -- level=1: access method type
    top_join text,       -- level>1: join method
    lhs integer,         -- level>1: pid of left child
    rhs integer,         -- level>1: pid of right child,
    cost float,          -- cost
    npages integer,      -- number of pages of output
    ntups integer,       -- number of tuples of output
    order_table text, 
    order_column integer,  -- this column and the preceding define the ordering key of the output
    parent integer,      -- pid of the parent node
    sibling_id integer,  -- position in order of children of this parent (1 = left, 2 = right),
    FOREIGN KEY (am_id) REFERENCES catalog_access_methods,
    FOREIGN KEY (top_join) REFERENCES system_join_methods,
    FOREIGN KEY (lhs) REFERENCES opt_bestpassplans,
    FOREIGN KEY (rhs) REFERENCES opt_bestpassplans,
    FOREIGN KEY (order_table, order_column) REFERENCES catalog_columns,
    FOREIGN KEY (parent) REFERENCES opt_bestpassplans,
    
    CHECK (   (level = 1 AND am_id IS NOT NULL AND type IS NOT NULL)
           OR (level > 1 AND top_join IS NOT NULL AND lhs IS NOT NULL and rhs IS NOT NULL))
);

INSERT INTO opt_bestpassplans
SELECT A.pid, A.level, ARRAY[table_name], A.am_id, A.type, NULL, NULL, NULL, 
       A.cost, A.npages, A.ntups, A.order_table, A.order_column, NULL, NULL
FROM opt_bestplans A;

Done.
Done.
7 rows affected.


[]

In [17]:
# minCostJoin: for pass i, we compare the cost for each join and choose the smallest one. Then we update the opt_bestpassplan
# table by inserting the output size, min cost and new table lists.
# 'output_size' = selectivity*output_size_fullscan, 'interesting_order',
# NEED TABLE: 'query_predicates', TABLE 'opt_bestplans', TABLE 'opt_bestpassplans'

# numpass: the # of passes need to compare i.e. two joins in where clause means two passses
def min_index(a):
    for ind in range(len(a)):
        if a[ind] == min(a):
            return ind

def minCostJoin(passCnt, left_pid, left_col, right_pid, right_col):
    # calculate 4 types of Joins and find the interesting order
    qbufs = %sql select qbufs from system_hardware;
    qbufs = qbufs[0][0]
    
    lhspages = %sql select npages from opt_bestpassplans where node_id = :left_pid;
    lhspages = lhspages[0][0]
    lhstups = %sql select ntups from opt_bestpassplans where node_id = :left_pid;
    lhstups = lhstups[0][0]
    left_order_column = %sql select order_column from opt_bestpassplans where node_id = :left_pid;
    left_order_column = left_order_column[0][0]
    
    rhspages = %sql select npages from opt_bestplans where pid = :right_pid;
    rhspages = rhspages[0][0]
    rhstups = %sql select ntups from opt_bestplans where pid = :right_pid;
    rhstups = rhstups[0][0]
    right_order_column = %sql select order_column from opt_bestpassplans where node_id = :right_pid;
    right_order_column = right_order_column[0][0]
    
    rhs_numvals = %sql SELECT num_vals \
                         FROM catalog_columns A \
                        WHERE A.column_id = :right_col AND \
                              A.table_name IN (SELECT table_name \
                                               FROM opt_bestplans \
                                               WHERE pid = :right_pid \
                                              );
    rhs_numvals = rhs_numvals[0][0];
    

    left_table_name = %sql SELECT tables FROM opt_bestpassplans WHERE node_id = :left_pid;
    right_table_name = %sql SELECT table_name FROM opt_bestplans WHERE pid = :right_pid;
    tables = left_table_name[0][0]
    tables.append(right_table_name[0][0])
    rightName = right_table_name[0][0]
    
    # cost of sort for both side - if there is interesting order, then cost is zero; otherwise we do quick sort
    leftSortCost = %sql SELECT SortCost(:lhstups, :left_order_column);
    rightSortCost = %sql SELECT SortCost(:rhstups, :right_order_column);
    sortCost = leftSortCost[0][0] + rightSortCost[0][0]
    
    # Get cost of all kind of joins ans select the cheapest one
    hashJoinCost = %sql SELECT HashJoinCost(:qbufs, :lhspages, :lhstups, :rhspages, :rhstups);
    hashJoinCost = hashJoinCost[0][0]
    sortMergeJoinCost =  %sql SELECT SortMergeJoinCost(:qbufs, :lhspages, :lhstups, :rhspages, :rhstups);
    sortMergeJoinCost = sortMergeJoinCost[0][0]
    BNLJCost = %sql SELECT BNLJCost(:qbufs, :lhspages, :lhstups, :rhspages, :rhstups);
    BNLJCost = BNLJCost[0][0]
    INLJCost = %sql SELECT INLJCost(:qbufs, :lhspages, :lhstups, :rhspages, :rhstups, :rhs_numvals);
    INLJCost = INLJCost[0][0]
    
    
    costType = ['Hash','Merge','BNLJ','INLJ']
    allCosts = [hashJoinCost, sortMergeJoinCost + sortCost, BNLJCost, INLJCost]
    
    # retain the minCost joins types & the interesting order AMONG ALL rows where level = passCnt
    # cost is the accumulative cost which equals the cost in the current level plus the left and right cost
    left_cost = %sql SELECT cost::decimal FROM opt_bestpassplans WHERE node_id = :left_pid;
    right_cost = %sql SELECT cost::decimal FROM opt_bestpassplans WHERE node_id = :right_pid;
    left_cost = left_cost[0][0]
    right_cost = right_cost[0][0]
    minCost = min(allCosts) + right_cost + left_cost
    top_join_min = costType[min_index(allCosts)]
   
    minINLJCost = INLJCost + right_cost + left_cost
    # calculate the output_size for npages and ntups; we use ceiling for the product by adding 1 
    twoColumnEqSel = %sql SELECT TwoColumnEqSelectivity(:rhs_numvals, :rhs_numvals);
    twoColumnEqSel = twoColumnEqSel[0][0]
    npages = int(float(lhspages)*float(rhspages)*float(twoColumnEqSel)+1.0)
    ntups = int(float(lhstups)*float(rhstups)*float(twoColumnEqSel)+1.0)
    
    #3 is the fourth element in the costType array, min_index!=3 means when it is not INLJ
    if min_index != 3:
        %sql INSERT INTO opt_bestpassplans \
             VALUES (nextval('anc_am_seq'), :passCnt, array_sort(:tables), NULL, NULL, :top_join_min, :left_pid \
                     , :right_pid, :minCost, :npages, :ntups, NULL, NULL, NULL, NULL);
        %sql INSERT INTO opt_bestpassplans \
             VALUES (nextval('anc_am_seq'), :passCnt, array_sort(:tables), NULL, NULL, 'INLJ', :left_pid \
                     , :right_pid, :minINLJCost, :npages, :ntups, :rightName, :left_col, NULL, NULL);
    elif min_index == 3:
        %sql INSERT INTO opt_bestpassplans \
             VALUES (nextval('anc_am_seq'), :passCnt, array_sort(:tables), NULL, NULL, 'INLJ', :left_pid \
                     , :right_pid, :minINLJCost, :npages, :ntups, :rightName, :left_col, NULL, NULL);
    


In [18]:
#The first for loop is to iterate each pass
#The second for loop is to iterate each plan for the current pass
#The third for loop is to iterate each join in the "query_join_predicates" table we constructed
#Inside the for loop: check left side and right side if there is join table name exists in the plan.
#If neither or both contains, we skip this join. If one of the sides contains, fetch the table for the other side.
#Then we call minCostJoin we created 

allJoin = %sql select pred_id from query_join_predicates;
numpasses = %sql select count(*) from query_join_predicates;
for passCnt in range(numpasses[0][0]):
    allPlan = %sql select node_id from opt_bestpassplans where level = :passCnt + 1; 
    for planCnt in allPlan:
        pidPlan = planCnt[0]
        singlePlan = %sql select * from opt_bestpassplans where node_id = :pidPlan;
        for joinCnt in allJoin:
            pidJoin = joinCnt[0]
            singleJoin = %sql select * from query_join_predicates where pred_id = :pidJoin;
            colLeft = singleJoin[0][2]
            colRight = singleJoin[0][4]
            hasLeft = singleJoin[0][1] in singlePlan[0][2]
            hasRight = singleJoin[0][3] in singlePlan[0][2]
            if ((hasLeft and hasRight) or (not hasLeft and not hasRight)):
                continue
            elif hasLeft:
                left_pid = pidPlan
                left_col = colLeft
                rightName = singleJoin[0][3]
                right_col = colRight
            else:
                left_pid = pidPlan
                left_col = colRight
                rightName = singleJoin[0][1]
                right_col = colLeft
            # right must choose from opt_bestplans
            pidRightPlan = %sql select pid from opt_bestplans where table_name = :rightName;
            for rightCnt in pidRightPlan:
                right_pid = rightCnt[0]
                minCostJoin(passCnt+2, left_pid, left_col, right_pid, right_col)


2 rows affected.
1 rows affected.
7 rows affected.
1 rows affected.
1 rows affected.
2 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
2 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affecte

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
48 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
2 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affect

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
2 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affecte

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
2 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
2 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affecte

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
2 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affecte

2 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
2 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affecte

### Now that we're done with iteration to find best join trees, let's choose the 1 best plan

In [19]:
%%sql
-- Pick the root of the cheapest plan. If there are ties on cost, break arbitrarily
DROP VIEW IF EXISTS opt_final;
CREATE VIEW opt_final AS (
-- YOUR CODE GOES HERE

-- IF there are two same cheapest plan, we break tie by selecting the smallest node_id number    

SELECT A.nid, B.level, B.tables, B.am_id, B.type, B.top_join, B.lhs, B.rhs, B.cost, B.npages, B.ntups, B.order_table, B.order_column, B.parent, B.sibling_id 
FROM (SELECT node_id as nid, tables, cost 
FROM opt_bestpassplans
WHERE level = (select count(*) from query_join_predicates) + 1
ORDER BY cost ASC, node_id ASC
LIMIT 1) AS A, opt_bestpassplans AS B
WHERE A.nid = B.node_id
);

Done.
Done.


[]

Now given the root of the cheapest plan, construct the full plan tree with its
parent-child relationships, and sibling orders.  Start from the root and work downward.

This can be done with a single, relatively simple [`WITH RECURSIVE`](https://www.postgresql.org/docs/9.6/static/queries-with.html) query, or you can use a Python loop 
to issue multiple queries.

In [20]:
#We recusively retrieve the whole tree given the root plan and its pid and store the pid for each node in the list
final_plan_ind = []
def fetch_pid(pid):
    final_plan_ind.append(pid)
    level_pid = %sql select level from opt_bestpassplans where node_id = :pid;
    level_pid = level_pid[0][0]
    if level_pid == 1:
        # do nothing
        level_pid = level_pid
    else:
        lhs_pid = %sql select lhs from opt_bestpassplans where node_id = :pid;
        fetch_pid(lhs_pid[0][0])
        rhs_pid = %sql select rhs from opt_bestpassplans where node_id = :pid;
        fetch_pid(rhs_pid[0][0])
root_pid = %sql select nid from opt_final;
fetch_pid(root_pid[0][0])

#INSERT INTO opt_chosen_plan
#SELECT * from opt_final;
final_plan_ind = sorted(final_plan_ind)
print final_plan_ind




1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
[3, 7, 8, 45, 203]


At this point, the best plan is stored in `opt_best_plan`, and the database system can compile it into iterator objects to execute. You can use the [visualizer notebook](visualizer.ipynb) to see a graphical version of the plan.

In [21]:
#With the list of pids, we store these nodes into opt_chosen_plan tables
for pid in final_plan_ind:
    %sql INSERT INTO opt_chosen_plan \
    SELECT * \
    FROM opt_bestpassplans \
    WHERE node_id = :pid;

1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


In [22]:
#This function is to recursively update the sibling and parent information for each node in the opt_chosen_plan table
def update_parent_sibling(pid):
    
    level_pid = %sql select level from opt_bestpassplans where node_id = :pid;
    level_pid = level_pid[0][0]
    if level_pid == 1:
        # do nothing
        level_pid = level_pid
    else:
        lhs_pid = %sql select lhs from opt_bestpassplans where node_id = :pid;
        rhs_pid = %sql select rhs from opt_bestpassplans where node_id = :pid;
        lhs_pid = lhs_pid[0][0]
        rhs_pid = rhs_pid[0][0]
        %sql UPDATE opt_chosen_plan\
             SET parent = :pid, sibling_id = :rhs_pid\
             WHERE node_id = :lhs_pid;
        %sql UPDATE opt_chosen_plan\
             SET parent = :pid, sibling_id = :lhs_pid\
             WHERE node_id = :rhs_pid;   
        update_parent_sibling(lhs_pid)
        update_parent_sibling(rhs_pid)
        
update_parent_sibling(root_pid[0][0])


1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


In [23]:
%%sql
--This is the final plan
select * from opt_chosen_plan;

5 rows affected.


node_id,level,tables,am_id,type,top_join,lhs,rhs,cost,npages,ntups,order_table,order_column,parent,sibling_id
203,3,"[u'boats', u'reserves', u'sailors']",None,None,Hash,45,3,751.852,1,667,None,None,None,None
45,2,"[u'boats', u'reserves']",None,None,INLJ,7,8,743.003,5,1710,None,None,203,3
3,1,[u'sailors'],1,heap,None,None,None,1.1,1,39,None,None,203,45
7,1,[u'boats'],5,btree,None,None,None,1.001,1,2,None,None,45,8
8,1,[u'reserves'],3,heap,None,None,None,641.0,541,100000,None,None,45,7


In [24]:
%%sql
select * from opt_bestpassplans;

247 rows affected.


node_id,level,tables,am_id,type,top_join,lhs,rhs,cost,npages,ntups,order_table,order_column,parent,sibling_id
8,1,[u'reserves'],3,heap,None,None,None,641.0,541,100000,None,None,None,None
5,1,[u'reserves'],6,btree,None,None,None,100100.0,541,100000,reserves,2,None,None
6,1,[u'reserves'],7,btree,None,None,None,100100.0,541,100000,reserves,3,None,None
1,1,[u'sailors'],4,btree,None,None,None,100.1,1,39,sailors,1,None,None
3,1,[u'sailors'],1,heap,None,None,None,1.1,1,39,None,None,None,None
7,1,[u'boats'],5,btree,None,None,None,1.001,1,2,None,None,None,None
2,1,[u'boats'],5,btree,None,None,None,117.117,1,2,boats,1,None,None
9,2,"[u'reserves', u'sailors']",None,None,Hash,8,1,1383.139,6,39001,None,None,None,None
10,2,"[u'reserves', u'sailors']",None,None,INLJ,8,1,542382.139,6,39001,sailors,2,None,None
11,2,"[u'reserves', u'sailors']",None,None,Hash,8,3,1284.139,6,39001,None,None,None,None
